# Numerically solving algebraic equations in Python

In this notebook, we demonstrate how to use Python to solve sets of equations.  Note that these equations can be non-linear, and as a result, there is the possibility that they can can no solutions or multiple solutions.

Let's try to solve the equation
\begin{align*}
x = e^{-x}
\end{align*}
The first thing we need to do is to transform this equation into a form $f(x)=0$.
\begin{align*}
x - e^{-x} = 0
\end{align*}
If we define the function $f(x)=x - e^{-x}$, we see that for a general value of $x$, the function is not equal to zero.  Let's see what this looks like on a plot.


In [ ]:
import pylab as plt
import numpy as np

def f(x):
    return x - np.exp(-x)


x_data = np.arange(0.0, 2.0, 0.01)
y_data= [f(x) for x in x_data]
plt.plot(x_data, y_data)

plt.xlabel(r'$x$')
plt.ylabel(r'$f(x)$')

plt.show()

In [ ]:
import numpy as np
from scipy.optimize import fsolve

def residual(x):
    return x - np.exp(-x)

x0 = 1.0
solution = fsolve(residual, x0)

print(solution)

Now let's explore the solution of systems of equations.  Consider the following:
\begin{align*}
y &= x
\\
x^2 + y^2 &= 1
\end{align*}
Graphically, this corresponds to an intersection between a line with a slope of $1$ and a unit circle.

In [ ]:
from scipy.optimize import fsolve

def residual(x):
    
    f = []
    f.append( x[0] - x[1] )
    f.append( x[0]**2 + x[1]**2 - 1.0)
    
    return f

x0 = [1.0, 0.0]
solution = fsolve(residual, x0)
print(solution)


In [ ]:
from pylab import *
from numpy import *


x_data = arange(-2.0, 2.0, 0.01)
y_data = [x for x in x_data]
plot(x_data, y_data, color='red')

x_data = arange(-1.0, 1.0, 0.001)
y_data = [np.sqrt(1.0-x**2) for x in x_data]
plot(x_data, y_data, color='green')
y_data = [-np.sqrt(1.0-x**2) for x in x_data]
plot(x_data, y_data, color='green')

ax = gca() #you first need to get the axis handle
ax.set_aspect(1) #sets the height to width ratio to 1.5.
show()




## CP101 Example 7.5

Consider the gaseous phase reaction:
\begin{align*}
{\rm  N_2 (g) + 3 H_2 (g) \leftrightarrows 2 NH_3 (g) }
\end{align*}

1.00 mol of nitrogen and 3.00 mol of hydrogen are fed to a reactor. The overall pressure is 1
atm. Calculate the equilibrium yield of ammonia at 298 K and 400 K.
Assume that enthalpy of reaction does not vary significantly within the temperature range under
consideration.

Thermochemical properties

|            |  $\Delta H_f$ |  $\Delta G_f$ |
|:---| ---:| ---:| 
|            | kJ mol$^{-1}$ | kJ mol$^{-1}$ |
| N$_2$ (g)  |          $0.00$ |          $0.00$ |
| H$_2$ (g)  |          $0.00$ |          $0.00$ |
| NH$_3$ (g) |        $-46.11$ |        $-16.63$ |


### Input data

First we input the necessary data for the problem.  Dictionaries will be used to hold the data.

In [ ]:
rxn = {'N2':-1.0, 'H2':-3.0, 'NH3':2.0}

molecule = {}
molecule['N2'] = {'Hf':0.0, 'Gf':0.0, 'N0':1.0}
molecule['H2'] = {'Hf':0.0, 'Gf':0.0, 'N0':3.0}
molecule['NH3'] = {'Hf':-46.11, 'Gf':-16.63, 'N0':0.0}

R = 8.314e-3  # kJ mol^{-1} K^{-1}
T = 298.15    # K
T0 = 298.15
p = 1.0

In [ ]:
import numpy as np
from scipy.optimize import fsolve

### Equilibrium constant

First we calculate the equilibrium constant.  This is related to the Gibbs free energy of reaction by
\begin{align*}
\ln K_p(T) &= \frac{\Delta G_{\rm rxn}(T)}{RT}
\end{align*}

The temperature dependence of the Gibbs free energy of reaction is given approximately by
\begin{align*}
\frac{\Delta G_{\rm rxn}(T)}{RT}
\approx \frac{\Delta G_{\rm rxn}(T_0)}{RT_0}
+ \frac{\Delta H_{\rm rxn}(T_0)}{RT_0} 
  \left(\frac{T_0}{T}-1\right)
\end{align*}


In [ ]:
H_rxn = 0.0
G_rxn = 0.0
for m, nu in rxn.items():
    H = molecule[m]['Hf']
    G = molecule[m]['Gf']
    H_rxn += nu*H
    G_rxn += nu*G

print(H_rxn)
print(G_rxn)
    
Kp = np.exp(-G_rxn/(R*T0) - (T0/T-1.0)*H_rxn/(R*T0))

def get_Kp(T):
    Kp = np.exp(-G_rxn/(R*T0) - H_rxn*(1.0/T-1.0/T0))
    return Kp

print(Kp)

In [ ]:
def residual(x, params):
    
    alpha = x
    Kp = params
    
    N = 0.0
    nu_sum = 0.0
    for m, nu in rxn.items():
        N0 = molecule[m]['N0']
        Nm = N0 + nu*alpha
        molecule[m]['moles'] = Nm
        N += Nm
        nu_sum += nu
    
    prod = 1.0
    for m, nu in rxn.items():
        y = molecule[m]['moles'] / N
        #print(m, y)
        prod *= y**nu
    prod *= p**nu_sum
        
    return prod - Kp

In [ ]:
alpha0 = 0.96
params = Kp

solution = fsolve(residual, alpha0, params)

print(solution)

In [ ]:
from scipy.optimize import bisect

slop= 1.0e-6
solution = bisect(residual, slop, 1.0-slop, args=(params,))

print(solution)

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html

In [ ]:
alpha0 = 0.96

T_data =  np.arange(300, 800, 10)

alpha_data = []
for T in T_data:
    params = get_Kp(T)
    solution = fsolve(residual, alpha0, params)
    
    alpha0 = solution[0]
    alpha_data.append(alpha0)
    
    
import pylab as plt

plt.plot(T_data, alpha_data)


plt.show()